In [1]:
##  Parallel processing & retries with original prompt
import os
import io
import json
import pandas as pd
from dotenv import load_dotenv
import PyPDF2
from azure.storage.blob import ContainerClient
from openai import AzureOpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load environment variables
load_dotenv()

# Set up Azure Blob Storage connection
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
container_name = "earningcalltranscriptscontainer"  # Replace with your container name
container_client = ContainerClient.from_connection_string(
    conn_str=connection_string,
    container_name=container_name
)

# Function to extract text from a PDF file stream
def extract_text_from_pdf(file_stream):
    text = ""
    try:
        pdf_reader = PyPDF2.PdfReader(file_stream)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text

# Function to truncate text to fit within the token limit
def truncate_text(input_text, max_tokens):
    tokens = input_text.split()  # Naive tokenization by splitting on spaces
    if len(tokens) > max_tokens:
        truncated_text = ' '.join(tokens[:max_tokens])
        return truncated_text
    return input_text

# Function to interact with Azure OpenAI and generate JSON output
def get_json(file_stream):
    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version="2024-02-01"
    )

    # Extract text from PDF
    input_context = extract_text_from_pdf(file_stream)

    # Truncate the input context to avoid exceeding token limit
    max_input_tokens = 12000  # Set max input tokens (adjust as needed)
    input_context = truncate_text(input_context, max_input_tokens)

    # System and user instructions for the OpenAI model
    instructions = """
     As a Financial Analyst, you will leverage your expertise to generate tailored Financial Analysis 
     Reports that cater to the specific requirements of clients. 
    Key Objectives:

    Extract and Present: Provide detailed figures for each metric listed above, ensuring accurate and up-to-date data.
    Categorize Clearly: Organize the metrics under their respective categories as outlined.
    Ensure Completeness: Verify that all relevant metrics are included and presented with sufficient context for accurate interpretation.
    Highlight Capital Expenditure: Ensure that capital expenditure-related metrics are prominently detailed and clearly separated from other categories.
    """
    
    user_content = f"""
    To extract and organize financial metrics from the report, including those related to capital expenditure, profitability, liquidity, solvency, cash flow, and other key indicators. The metrics should be categorized and detailed according to their relevance to different aspects of financial analysis. 

    Operating Activities "Change in Working Capital",
            "Net Cash from Operating Activities"
    Investing Activities has "Acquisition of Fixed Assets & Intangibles",
            "Net Cash from Investing Activities"
    Financing Activities includes "Dividends Paid",
            "Cash from (Repayment of) Debt",
            "Net Cash from Financing Activities",
    Net Change includes  Net Change in Cash
    Metadata includes "Report Date", "Publish Date", "Source"
    Profitability Metrics like "EBITDA",
            "Gross Profit Margin",
            "Operating Margin",
            "Net Profit Margin",
            "Return on Equity",
            "Return on Assets",
            "Return On Invested Capital",
    Liquidity Metrics include Current Ratio
    Solvency Metrics like  "Total Debt", "Liabilities to Equity Ratio", "Debt Ratio",
    Cash Flow Metrics like "Free Cash Flow", "Free Cash Flow to Net Income", "Cash Return On Invested Capital",
    Other Important Metrics like "Piotroski F-Score", "Net Debt / EBITDA", "Dividend Payout Ratio
    Capital Expenditure 
    ```Document
    {input_context}
    ```
    """

    # JSON output structure request
    final_content = """
    Please provide the company name, year and quarter, capex value in billions of US dollars and no need of use $ symbol 
    in JSON format.
    Use the following JSON structure:
    ```json
    {{
        "company": "",
        "year": "",
        "quarter": "",
        "capex": ""    
    }}
    ```
    Note: Only JSON Data is required, no other text is required.
    """

    # Steps to find the capital expenditure value
    how_content = """
    Please provide the steps how you found or calculated the capital expenditure value.
    """

    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": user_content}
    ]

    try:
        # Initial completion request
        response = client.chat.completions.create(
            model=os.getenv("DEPLOYMENT_NAME"),
            messages=messages,
        )
        response_content = response.choices[0].message.content
        messages.append({"role": "assistant", "content": response_content})
        messages.append({"role": "user", "content": final_content})
    except Exception as e:
        print(f"Error during OpenAI response generation: {e}")
        return None, None

    try:
        # Requesting JSON formatted data
        response_json = client.chat.completions.create(
            model=os.getenv("DEPLOYMENT_NAME"),
            messages=messages,
        )
        messages.append({"role": "assistant", "content": response_json.choices[0].message.content})
        messages.append({"role": "user", "content": how_content})

        # Requesting explanation of how the value was found
        how_response = client.chat.completions.create(
            model=os.getenv("DEPLOYMENT_NAME"),
            messages=messages
        )
    except Exception as e:
        print(f"Error during JSON extraction or explanation generation: {e}")
        return None, None

    return response_json.choices[0].message.content, how_response.choices[0].message.content

# Function to process a single blob with retry logic
def process_blob(blob, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            stream = io.BytesIO()
            container_client.download_blob(blob).readinto(stream)
            op, how = get_json(stream)
            if op and how:
                op = json.loads(op[op.index("{"):len(op)-op[::-1].index("}")])
                if op.get("capex"):  # Check if capex is not empty
                    return op, how
                else:
                    print(f"Capex is empty. Retrying for blob {blob.name}...")
            attempt += 1
            time.sleep(5)  # Wait before retrying
        except Exception as e:
            print(f"Error processing blob {blob.name}: {e}")
            return None, None
    print(f"Max retries reached for blob {blob.name}.")
    return None, None

# Main function to process blobs in parallel
def main():
    final_json = []
    blob_list = list(container_client.list_blobs())[:]  # Limit to the first 20 blobs

    # Using ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=50) as executor:
        future_to_blob = {executor.submit(process_blob, blob): blob for blob in blob_list}
        # Iterate over completed futures
        for future in as_completed(future_to_blob):
            blob = future_to_blob[future]
            try:
                result = future.result()
                if result:
                    op, how = result
                    if op:  # Ensure op is not None
                        print(op, "\n\n", how)
                        final_json.append(op)
            except Exception as e:
                print(f"Error processing result for blob {blob.name}: {e}")

    # Ensure final_json contains valid data
    final_json = [item for item in final_json if isinstance(item, dict)]
    # Optional: Save final JSON to a file
    output_file = "financial_analysis_results_1201.json"
    with open(output_file, "w") as outfile:
        json.dump(final_json, outfile, indent=4)

    # Convert final_json to DataFrame and save to CSV
    if final_json:  # Check if final_json is not empty
        df = pd.DataFrame(final_json)
        output_file = "financial_analysis_results10.csv"
        df.to_csv(output_file, index=False)
        print(f"Processing completed. Results saved to {output_file}.")
    else:
        print("No valid data to save to CSV.")

if __name__ == "__main__":
    main()


{'company': 'Berkshire Hathaway Inc.', 'year': '2018', 'quarter': 'Q2', 'capex': 'N/A'} 

 Apologies, but it seems that the capital expenditure value is not mentioned in the provided report.
{'company': 'Berkshire Hathaway Inc.', 'year': '2014', 'quarter': 'Q2', 'capex': 'N/A'} 

 Based on the information provided in the financial report, the specific value for capital expenditure (capex) is not mentioned. The report does not provide any details regarding capital expenditure for Berkshire Hathaway Inc. in the given quarter and year. Therefore, I am unable to calculate or provide a specific value for this metric.
{'company': 'Berkshire Hathaway Inc.', 'year': '2014', 'quarter': 'Q3', 'capex': '0'} 

 From the given report, there is no specific mention of the capital expenditure value. Therefore, it is not possible to calculate or find the capital expenditure value from the provided information.
{'company': 'Berkshire Hathaway Inc.', 'year': '2017', 'quarter': 'Q3', 'capex': 'Not Provide

Incomplete download: ('Connection broken: IncompleteRead(146788 bytes read, 1265 more expected)', IncompleteRead(146788 bytes read, 1265 more expected))


{'company': 'Berkshire Hathaway Inc.', 'year': '2022', 'quarter': 'Q2', 'capex': '1.0'} 

 To find the capital expenditure value, I would follow these steps:

1. Review the financial report provided and identify the section that provides information on capital expenditure.
2. Look for line items or descriptions that specifically mention capital expenditure, such as "Acquisition of Fixed Assets" or "Capital Expenditure".
3. Take note of the value associated with the capital expenditure category.
4. If the value is provided in a different unit (e.g., millions or thousands), convert it to billions of US dollars.
5. Include the formatted value in the JSON structure.
{'company': 'Alphabet Inc.', 'year': 2021, 'quarter': 'Q4', 'capex': 6.383} 

 The capital expenditure value can be found in the provided report under the "Investing Activities" section. Specifically, we can look for the line item "Purchases of Property and Equipment" which represents the capital expenditures made by the compan

Incomplete download: ('Connection broken: IncompleteRead(146788 bytes read, 1265 more expected)', IncompleteRead(146788 bytes read, 1265 more expected))


{'company': 'Cardinal Health, Inc.', 'year': '2016', 'quarter': 'Q2', 'capex': 'N/A'} 

 I apologize for the confusion, but the provided text does not contain any specific information or figures related to capital expenditure. Without any specific data or calculation provided in the text, it is not possible to determine or calculate the capital expenditure value. If you have the relevant financial information or figures related to capital expenditure, please provide them so I can assist you in calculating the value.
{'company': 'Cardinal Health, Inc.', 'year': 'FY17', 'quarter': 'Q1', 'capex': '2.3'} 

 Based on the given document, the specific value for capital expenditure was not mentioned. Therefore, without the specific information, it is not possible to calculate the capital expenditure value.
{'company': 'Cardinal Health, Inc.', 'year': '2016', 'quarter': 'Q3', 'capex': '0.45'} 

 Unfortunately, the
{'company': 'Cardinal Health, Inc.', 'year': '2017', 'quarter': 'Q3', 'capex': 0.

Incomplete download: ('Connection broken: IncompleteRead(146788 bytes read, 1265 more expected)', IncompleteRead(146788 bytes read, 1265 more expected))


{'company': 'Chevron Corporation', 'year': '2020', 'quarter': 'Q3', 'capex': '10.3'} 

 Based on the provided financial report from Chevron Corporation, the capital expenditure value for the first nine months of 2020 was mentioned in the report as $10.3 billion. This information was directly extracted from the report and no further calculations or estimations were required.
Error processing blob 2022q4-alphabet-earnings-release.pdf: ('Connection broken: IncompleteRead(146788 bytes read, 1265 more expected)', IncompleteRead(146788 bytes read, 1265 more expected))
{'company': 'Chevron Corporation', 'year': '2021', 'quarter': 'Q1', 'capex': '2.5'} 

 As the provided financial report did not explicitly state the capital expenditure value, I was unable to find or calculate it. However, if you have access to additional information or data related to Chevron Corporation's capital expenditure, please provide it so that I can assist you further in calculating the capital expenditure value.
{'co

In [6]:
##Cleaning the data 
import pandas as pd

# Read the original CSV file
input_file = "financial_analysis_results10.csv"
df = pd.read_csv(input_file)
df['company'] = df['company'].replace({
    'Amazon': 'Amazon.com',
    'Amazon.com, Inc.': 'Amazon.com',
    'Alphabet Inc':'Alphabet Inc.',
    'Amazon.com Inc.': 'Amazon.com',
    'Goldman Sachs' : 'Goldman Sachs Group, Inc.',
    'The Goldman Sachs Group, Inc.': 'Goldman Sachs Group, Inc.',
    'Goldman Sachs Group':'Goldman Sachs Group, Inc.',
    'Tesla' : 'Tesla Inc.',
    'Tesla, Inc.':'Tesla Inc.',
    'Tesla Motors, Inc.': 'Tesla Inc.',
    'Tesla Motors':'Tesla Inc.',
    'Valero Energy Corporation': 'Valero Energy',
    'Cardinal Health':'Cardinal Health, Inc.',
    'BERKSHIRE HATHAWAY INC.': 'Berkshire Hathaway Inc.',
    'Berkshire Hathaway Inc' : 'Berkshire Hathaway Inc.',
    '"Cardinal Health, Inc."':'Cardinal Health, Inc.',
    'Bank of America Corporation': 'Bank of America',
    'CVS Health Corp': 'CVS Health Corporation',
    'CVS Health':'CVS Health Corporation',
    'Cigna Corporation':'CIGNA', 
    'Cigna Corp.':'CIGNA', 
    'Cigna Corp':'CIGNA',
      'CI':'CIGNA',
      'CIGNA Corporation':'CIGNA',
      'Cigna':'CIGNA', 
      'Cigna Group':'CIGNA',
      'The Cigna Group':'CIGNA', 
      'Citigroup Inc.':'Citi',
      'Citigroup Inc':'Citi',
      'CMCSA':'Comcast',
      'Comcast Corporation':'Comcast',
      'Comcast Corp':'Comcast',
      'WellPoint':'Wellpoint Inc',
      'WellPoint':'Wellpoint Inc.',
      'Anthem Inc': 'Anthem Inc.', 
    'Elevance Health':'Elevance Health Inc', 
    'Exxon Mobil Corporation':'ExxonMobil',
      'ExxonMobil Corporation':'ExxonMobil',
       'Ford Motor Company':'Ford Motor', 
       'Ford Motor Co':'Ford Motor',
       'Ford Motor Co.':'Ford Motor',
        'Humana Inc':'Humana Inc.',
       'JPMorgan Chase & Co':'JPMorgan Chase & Co.',
       'JPMorgan Chase':'JPMorgan Chase & Co.',
       'McKesson Corporation':'McKesson' , 
       'McKESSON':'McKesson',
       'McKESSON CORPORATION':'McKesson',  
       'McKesson Corp.':'McKesson',
       'McKesson Corp':'McKesson',
       'Walgreens Boots Alliance':'Walgreens Boots Alliance Inc.',
       'Walgreens Boots Alliance Inc':'Walgreens Boots Alliance Inc.', 
       'Wal-Mart Stores, Inc.':'Walmart', 
      'WAL-MART STORES, INC':'Walmart',
       'Walmart Inc.':'Walmart', 
       'WAL-MART STORES, INC.':'Walmart',
       'WAL-MART STORES, INC. (NYSE: WMT)':'Walmart', 
       'Walmart, Inc.':'Walmart',
      
})

df['capex'] = df['capex'].replace({ 'Not provided in the report':'-', 'nan':'-' ,'Not provided': '-', 'not available':'-','To be determined':'-','[Data Not Provided]':'-','Not Provided': '-','Not mentioned in the report':'-','X':'-', 'Not available': '-',  'Not specified': '-', 'n/a': '-', 'nan': '-','X.XX': '-','Not provided in the Report':'-','Not specified in the provided document':'-','Not Provided in the Report':'-','Not available in the provided document':'-','Not Available' :'-','[figures not provided]':'-', 'Not specified in the text provided': '-','Not mentioned' : '-','Not disclosed':'-'})
df['company'] = df['company'].str.replace('"', ' ', regex=False)
df['year'] = df['year'].replace({'FY16' :'2016', 'FY21':'2021', 'FY20':'2020', 'FY19':'2019', 'FY23':'2023', 'FY24':'2024', 'FY2024':'2024', 'FY17':'2017', 'Q3 FY15':'2015', 'Q1 FY18' : '2018', 'Q1 FY21':'2021'})
df['quarter'] = df['quarter'].replace({'First Quarter': 'Q1', '2': 'Q2', 'Second Quarter': 'Q2', 'Third Quarter': 'Q3', '3rd Quarter':'Q3', '1':'Q1', '4th':'Q4', '2nd Quarter':'Q2', '2Q':'Q2', '3Q':'Q3', 'Fourth Quarter' : 'Q4','Fourth':'Q4','fourth':'Q4', 'Second':'Q2','4': 'Q4','First':'Q1', '4Q':'Q4','4':'Q4', 'Third':'Q3', '1Q':'Q1'})
# Save the rearranged DataFrame to a new CSV file
output_file = "cleaned_financial_analysis_results1.csv"
df.to_csv(output_file, index=False)

print(f"cleaned data saved to {output_file}.")


cleaned data saved to cleaned_financial_analysis_results1.csv.


In [7]:
## Rearranged input capex with quarter_year with company name 
import pandas as pd

# Read the original CSV file
input_file = "cleaned_financial_analysis_results1.csv"
df = pd.read_csv(input_file)

# Convert 'quarter' and 'year' columns into a single 'Quarter-Year' column with an underscore
df['Quarter-Year'] = df['quarter'] + "_" + df['year'].astype(str)

# Pivot the DataFrame
pivot_df = df.pivot_table(index='company', columns='Quarter-Year', values='capex', aggfunc='first')

# Reset the index to make 'company' a column instead of an index
pivot_df.reset_index(inplace=True)

# Define a function to sort columns in the desired order
def sort_columns(df):
    # Extract the current columns
    columns = df.columns.tolist()
    
    # Extract the company column and the Quarter-Year columns
    company_col = columns[0]
    quarter_cols = columns[1:]
    
    # Generate sorted columns list: Start with the most recent quarters
    sorted_quarters = sorted(quarter_cols, key=lambda x: (x.split('_')[1], x.split('_')[0]), reverse=True)
    
    # Combine sorted columns with the company column
    sorted_columns = [company_col] + sorted_quarters
    return sorted_columns

# Reorder the columns
sorted_columns = sort_columns(pivot_df)
pivot_df = pivot_df[sorted_columns]
pivot_df['company'].unique()
pivot_df['company'] = pivot_df['company'].str.replace('"', ' ', regex=False)
# # Save the rearranged DataFrame to a new CSV file
output_file = "rearranged_financial_analysis_results1.csv"
pivot_df.to_csv(output_file, index=False)

print(f"Rearranged data saved to {output_file}.")


Rearranged data saved to rearranged_financial_analysis_results1.csv.


In [8]:
import pandas as pd

# Read the CSV file
input_file = "rearranged_financial_analysis_results1.csv"
df = pd.read_csv(input_file)

# Define a function to compare capex values between consecutive columns
def compare_columns(row):
    comparisons = []
    for i in range(1, len(row) - 1):  # Skip the 'company' column
        current_value = row[i]
        next_value = row[i + 1]
        
        if pd.isna(current_value) or pd.isna(next_value):
            comparisons.append("Data not available")
        else:
            try:
                current_value = float(current_value)
                next_value = float(next_value)
                if next_value < current_value:
                    comparisons.append("increase")
                elif next_value > current_value:
                    comparisons.append("decrease")
                else:
                    comparisons.append("unchanged")
            except ValueError:
                comparisons.append("Data not available")
    
    return comparisons

# Apply the comparison function to each row
comparison_results = df.apply(lambda row: compare_columns(row), axis=1)

# Replace the original values in the DataFrame with the comparison results
for i, col in enumerate(df.columns[1:-1]):  # Exclude 'company' and the last quarter
    df[col] = comparison_results.apply(lambda x: x[i])

# Save the comparison results to a new CSV file
output_file = "capex_comparison_results.csv"
df.to_csv(output_file, index=False)

print(f"Capex comparison results saved to {output_file}.")


Capex comparison results saved to capex_comparison_results.csv.


/tmp/ipykernel_7771/3537409788.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_value = row[i]
/tmp/ipykernel_7771/3537409788.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  next_value = row[i + 1]
